In [4]:
# import modified visualpriors library
from transforms import VisualPriorRepresentation, VisualPriorPredictedLabel
from taskonomy_network import TaskonomyEncoder, TaskonomyDecoder

import torchsummary
from PIL import Image
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

import torchvision.transforms.functional as TF
import torch.utils.model_zoo # required to load nets
from torch import nn
import collections

import classes

%load_ext autoreload
%autoreload 2

# Load Taskonomy Network

In [2]:
task = 'autoencoding'

VisualPriorRepresentation._load_unloaded_nets([task]) # preload nets
net = VisualPriorRepresentation.feature_task_to_net[task] # loads encoder only for representations

In [ ]:
torchsummary.summary(net, (3, 224, 224))

# Create tweaked net for retrieving activations

In [3]:
train_nodes, eval_nodes = get_graph_node_names(net)
print(len(eval_nodes))
eval_nodes


180


['x',
 'pad',
 'conv1',
 'bn1',
 'relu',
 'maxpool',
 'layer1.0.conv1',
 'layer1.0.bn1',
 'layer1.0.relu',
 'layer1.0.conv2',
 'layer1.0.bn2',
 'layer1.0.relu_1',
 'layer1.0.conv3',
 'layer1.0.bn3',
 'layer1.0.downsample.0',
 'layer1.0.downsample.1',
 'layer1.0.add',
 'layer1.0.relu_2',
 'layer1.1.conv1',
 'layer1.1.bn1',
 'layer1.1.relu',
 'layer1.1.conv2',
 'layer1.1.bn2',
 'layer1.1.relu_1',
 'layer1.1.conv3',
 'layer1.1.bn3',
 'layer1.1.add',
 'layer1.1.relu_2',
 'layer1.2.conv1',
 'layer1.2.bn1',
 'layer1.2.relu',
 'layer1.2.conv2',
 'layer1.2.bn2',
 'layer1.2.relu_1',
 'layer1.2.conv3',
 'layer1.2.bn3',
 'layer1.2.downsample.0',
 'layer1.2.add',
 'layer1.2.relu_2',
 'layer2.0.conv1',
 'layer2.0.bn1',
 'layer2.0.relu',
 'layer2.0.conv2',
 'layer2.0.bn2',
 'layer2.0.relu_1',
 'layer2.0.conv3',
 'layer2.0.bn3',
 'layer2.0.downsample.0',
 'layer2.0.downsample.1',
 'layer2.0.add',
 'layer2.0.relu_2',
 'layer2.1.conv1',
 'layer2.1.bn1',
 'layer2.1.relu',
 'layer2.1.conv2',
 'layer2.1.b

In [4]:
return_nodes = { k:k for k in ['conv1']+[node for node in eval_nodes if "conv3" in node]}

In [5]:
net_tweaked = create_feature_extractor(net, return_nodes=return_nodes)
net_tweaked

TaskonomyEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (layer1): Module(
    (0): Module(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Module(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
   

# Load Image, convert to tensor, create image parts

In [ ]:
def checkerboard(scale, output_size=640):
    board = np.indices((scale,scale)).sum(axis=0) % 2
    board = board.repeat(output_size/scale,axis=0).repeat(output_size/scale, axis=1)
    board = board[:,:, np.newaxis,].repeat(3,axis=2)
    return board.astype(dtype = np.bool_)

In [ ]:
img = Image.open('./data/stimuli_places1/Places365_val_00036011.jpg')
img = img.resize((640,640))




In [ ]:
img = np.array(img)

img1, img2 = np.where(checkerboard(4), img, 255), np.where(~checkerboard(4), img, 255)

img, img1, img2 = Image.fromarray(img), Image.fromarray(img1), Image.fromarray(img2)
img, img1, img2 = TF.to_tensor(TF.resize(img, 256)) * 2 - 1, TF.to_tensor(TF.resize(img1, 256)) * 2 - 1, TF.to_tensor(TF.resize(img2, 256)) * 2 - 1
img, img1, img2 = img.unsqueeze_(0), img1.unsqueeze_(0), img2.unsqueeze_(0)
img.shape, img1.shape, img2.shape

# Get activations for image and image parts

In [ ]:
img_act = net_tweaked(img)
img1_act = net_tweaked(img1)
img2_act = net_tweaked(img2)

# Calculate average activation for image parts

In [ ]:
img12avg_act = { k:None for k in img_act.keys()}
img12avg_act.keys()

In [ ]:
for layer in img_act.keys():
    img12avg_act[layer] = torch.stack((img1_act[layer], img2_act[layer]), dim=0).mean(dim=0)

In [ ]:
print("Sanity check on average partial and full activations")
for layer in img_act.keys():
    print(img_act[layer].shape == img12avg_act[layer].shape)

# Integration coeficient

In [6]:
integration = { k:None for k in img_act.keys()}

for (layer_name, a1, a2) in zip(integration.keys(), img_act.values(), img12avg_act.values()):
    integration[layer_name] = pearsonr(a1.flatten(), a2.flatten())[0]

NameError: name 'img_act' is not defined

In [ ]:
integration

In [ ]:
plt.plot(list(integration.values()))

# Read beauty ratings


In [ ]:
pd.read_csv('~/Documents/Thesis/Taskonomy Integration/behavior/ratings_study1.csv', header=None).mean(axis=1)

In [ ]:
img_dir = '/home/max/Documents/Thesis/Taskonomy Integration/data/stimuli_places1'
l = list(f for f in os.listdir(img_dir) if os.path.isfile(os.path.join(img_dir, f)) and f.endswith('.jpg'))
l.sort()
l

In [ ]:
pl_dataset= classes.ImageDataset('/home/max/Documents/Thesis/Taskonomy Integration/data/stimuli_places1')
pl_it = iter(pl_dataset)

# Correlate integration with beauty

In [7]:
inc = classes.IntegrationCalculator(net, return_nodes)

In [11]:
pl_dataset= classes.ImageDataset('./data/stimuli_places1', beauty_ratings_path='./behavior/ratings_study1.csv')

In [14]:
l = []
for idx, img in enumerate(iter(pl_dataset)):
    l.append(inc.integration_coeff(img))
    if idx == 40:
        break

res = pd.DataFrame(l, columns=inc.evalutation_layers)

In [15]:
pl_dataset.beauty_ratings

0      86.00
1      70.76
2      33.68
3      35.84
4      40.72
       ...  
245    60.76
246    38.52
247    65.24
248    56.44
249    62.48
Length: 250, dtype: float64

In [16]:
res

,conv1,layer1.0.conv3,layer1.1.conv3,layer1.2.conv3,layer2.0.conv3,layer2.1.conv3,layer2.2.conv3,layer2.3.conv3,layer3.0.conv3,layer3.1.conv3,layer3.2.conv3,layer3.3.conv3,layer3.4.conv3,layer3.5.conv3,layer4.0.conv3,layer4.1.conv3,layer4.2.conv3
0,-0.901755,-0.892599,-0.839657,-0.852555,-0.861921,-0.892498,-0.857943,-0.870222,-0.898146,-0.941513,-0.842541,-0.556739,-0.938784,-0.859965,-0.725659,-0.366636,-0.371284
1,-0.977258,-0.970708,-0.918240,-0.947710,-0.951693,-0.961840,-0.950832,-0.931376,-0.974267,-0.978756,-0.928855,-0.932162,-0.954501,-0.938060,-0.805249,-0.470574,-0.136613
2,-0.985005,-0.978973,-0.937909,-0.965159,-0.972312,-0.971635,-0.966061,-0.954062,-0.982647,-0.980379,-0.946555,-0.998034,-0.966077,-0.932084,-0.840916,-0.497462,-0.148527
3,-0.969972,-0.959294,-0.924279,-0.948057,-0.943261,-0.951317,-0.948021,-0.935146,-0.969792,-0.969520,-0.827942,-0.918534,-0.951156,-0.903482,-0.862330,-0.582552,-0.420497
4,-0.982985,-0.975774,-0.950199,-0.963792,-0.965886,-0.967316,-0.965047,-0.952409,-0.979744,-0.973700,-0.855534,-0.806452,-0.941134,-0.902900,-0.884209,-0.626542,-0.499548
5,-0.898819,-0.884886,-0.849355,-0.865869,-0.859547,-0.879700,-0.866660,-0.873947,-0.895659,-0.936772,-0.858648,-0.791237,-0.957753,-0.869697,-0.766619,-0.394607,-0.383541
6,-0.985219,-0.979381,-0.956440,-0.971500,-0.974353,-0.974174,-0.970551,-0.958855,-0.980163,-0.981898,-0.886448,-0.822022,-0.961846,-0.873255,-0.880735,-0.577256,-0.297019
7,-0.981992,-0.975154,-0.939756,-0.959388,-0.964365,-0.967351,-0.958219,-0.946594,-0.978282,-0.974926,-0.904808,-0.989909,-0.955010,-0.930499,-0.871263,-0.543245,-0.262278
8,-0.978424,-0.973825,-0.955861,-0.959867,-0.964868,-0.971782,-0.967457,-0.960790,-0.981707,-0.968975,-0.837813,-0.897551,-0.954960,-0.868899,-0.919503,-0.750484,-0.695347
9,-0.975752,-0.971692,-0.937749,-0.955494,-0.953245,-0.956554,-0.954924,-0.948987,-0.971086,-0.978400,-0.891148,-0.973769,-0.963909,-0.913098,-0.882087,-0.547102,-0.257479


In [ ]:
res.aggregate(lambda x: pearsonr(x, ratings)[0], axis= 0).plot()

# Linear Modelling

In [21]:
"""
   X: Integration measures, DataFrame (num_images x num_layers)
   y: Beauty reatings, Series (num_images x 1)
"""
pl_dataset.beauty_ratings
res


,conv1,layer1.0.conv3,layer1.1.conv3,layer1.2.conv3,layer2.0.conv3,layer2.1.conv3,layer2.2.conv3,layer2.3.conv3,layer3.0.conv3,layer3.1.conv3,layer3.2.conv3,layer3.3.conv3,layer3.4.conv3,layer3.5.conv3,layer4.0.conv3,layer4.1.conv3,layer4.2.conv3
0,-0.901755,-0.892599,-0.839657,-0.852555,-0.861921,-0.892498,-0.857943,-0.870222,-0.898146,-0.941513,-0.842541,-0.556739,-0.938784,-0.859965,-0.725659,-0.366636,-0.371284
1,-0.977258,-0.970708,-0.918240,-0.947710,-0.951693,-0.961840,-0.950832,-0.931376,-0.974267,-0.978756,-0.928855,-0.932162,-0.954501,-0.938060,-0.805249,-0.470574,-0.136613
2,-0.985005,-0.978973,-0.937909,-0.965159,-0.972312,-0.971635,-0.966061,-0.954062,-0.982647,-0.980379,-0.946555,-0.998034,-0.966077,-0.932084,-0.840916,-0.497462,-0.148527
3,-0.969972,-0.959294,-0.924279,-0.948057,-0.943261,-0.951317,-0.948021,-0.935146,-0.969792,-0.969520,-0.827942,-0.918534,-0.951156,-0.903482,-0.862330,-0.582552,-0.420497
4,-0.982985,-0.975774,-0.950199,-0.963792,-0.965886,-0.967316,-0.965047,-0.952409,-0.979744,-0.973700,-0.855534,-0.806452,-0.941134,-0.902900,-0.884209,-0.626542,-0.499548
5,-0.898819,-0.884886,-0.849355,-0.865869,-0.859547,-0.879700,-0.866660,-0.873947,-0.895659,-0.936772,-0.858648,-0.791237,-0.957753,-0.869697,-0.766619,-0.394607,-0.383541
6,-0.985219,-0.979381,-0.956440,-0.971500,-0.974353,-0.974174,-0.970551,-0.958855,-0.980163,-0.981898,-0.886448,-0.822022,-0.961846,-0.873255,-0.880735,-0.577256,-0.297019
7,-0.981992,-0.975154,-0.939756,-0.959388,-0.964365,-0.967351,-0.958219,-0.946594,-0.978282,-0.974926,-0.904808,-0.989909,-0.955010,-0.930499,-0.871263,-0.543245,-0.262278
8,-0.978424,-0.973825,-0.955861,-0.959867,-0.964868,-0.971782,-0.967457,-0.960790,-0.981707,-0.968975,-0.837813,-0.897551,-0.954960,-0.868899,-0.919503,-0.750484,-0.695347
9,-0.975752,-0.971692,-0.937749,-0.955494,-0.953245,-0.956554,-0.954924,-0.948987,-0.971086,-0.978400,-0.891148,-0.973769,-0.963909,-0.913098,-0.882087,-0.547102,-0.257479


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut

In [64]:
X = StandardScaler().fit_transform(res)
y = StandardScaler().fit_transform(pd.DataFrame(pl_dataset.beauty_ratings)[0:40+1])
X.shape, y.shape

((41, 17), (41, 1))

In [68]:
loo = LeaveOneOut()
g = loo.split(X)

In [70]:
t, te = next(g)

In [75]:
X[t];

In [76]:
lr = LinearRegression().fit(X[t], y[t])

In [79]:
lr.predict(X[te]).item()

0.6172698321198075

In [80]:
loo = LeaveOneOut()
l = []
for train_idx, predict_idx in loo.split(X):
    glm = LinearRegression().fit(X[train_idx], y[train_idx])
    l.append(glm.predict(X[predict_idx]).item())

In [89]:
predictions = pd.Series(l)
predictions

0     0.357821
1     0.617270
2    -0.132017
3     1.681160
4     0.972193
5     2.408896
6     0.380957
7    -0.014207
8     0.719692
9    -1.067746
10    0.072012
11   -1.436126
12   -1.001225
13    2.612830
14    0.520758
15   -0.456783
16   -1.786309
17    0.245816
18    0.536903
19    0.352482
20   -1.763249
21   -0.719777
22    0.002106
23   -0.487726
24    0.533270
25   -0.188145
26   -0.961780
27    0.313153
28   -0.211780
29    0.420431
30    0.444290
31    0.681206
32    1.293144
33   -0.977474
34   -0.927567
35   -0.746145
36    0.631907
37   -0.006115
38   -0.674779
39    0.631685
40   -1.180153
dtype: float64

In [90]:
pl_dataset.beauty_ratings[0:40+1]

0     86.00
1     70.76
2     33.68
3     35.84
4     40.72
5     28.68
6     41.36
7     55.72
8     36.76
9     50.12
10    90.72
11    72.48
12    76.88
13    62.16
14    32.36
15    36.24
16    43.56
17    58.32
18    29.48
19    37.96
20    89.24
21    69.64
22    37.64
23    81.52
24    86.36
25    25.72
26    67.16
27    55.08
28    30.12
29    77.32
30    59.92
31    31.68
32    43.88
33    37.28
34    34.72
35    55.68
36    59.80
37    41.72
38    71.36
39    83.76
40    33.92
dtype: float64

In [93]:
pearsonr(predictions, pl_dataset.beauty_ratings[0:40+1])

(-0.1773209568062218, 0.26738156700988414)

# Checking implementation with vgg16_places net

## Load vgg16_places365 weights from converted caffe model

In [1]:
import caffemodel2pytorch as caffe

In [3]:
vgg16_places_net = caffe.Net(prototxt='deploy_vgg16_places365.prototxt', weights='vgg16_places365.caffemodel', caffe_proto='https://raw.githubusercontent.com/BVLC/caffe/master/src/caffe/proto/caffe.proto')

caffemodel2pytorch: loading model from [vgg16_places365.caffemodel] in HDF5 format failed [No module named 'h5py'], falling back to caffemodel format
caffemodel2pytorch: loaded model from [vgg16_places365.caffemodel] in caffemodel format


In [38]:
torchsummary.summary(vgg16_places_net, input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       Convolution-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
       Convolution-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
       Convolution-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
       Convolution-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
      Convolution-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
      Convolution-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [ ]:
vgg16_places_torchload = torch.load('vgg16_places365.caffemodel.pt')

In [ ]:
vgg16_places_torchload['conv1_1.weight'].shape

In [ ]:
# get parameter names (need to match those in model definition)
list(name for name, _ in vgg16_places_torchload.named_parameters())

## Inspect a vanilla vgg16 network

In [ ]:
import torchvision.models
vgg16_vanilla = torchvision.models.vgg16()
vgg16_vanilla.features = torch.nn.Sequential(collections.OrderedDict(zip(['conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1', 'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2', 'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3', 'relu3_3', 'pool3', 'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3', 'relu4_3', 'pool4', 'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3', 'relu5_3', 'pool5'], vgg16_vanilla.features)))
vgg16_vanilla.classifier = torch.nn.Sequential(collections.OrderedDict(zip(['fc6', 'relu6', 'drop6', 'fc7', 'relu7', 'drop7', 'fc8'], vgg16_vanilla.classifier)))

In [28]:
torchsummary.summary(vgg16_vanilla, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## Initialise vgg16_places365 architecture

In [125]:
vgg16_places = nn.Sequential(
    collections.OrderedDict(
        [
            ('conv1_1', nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)),
            ('ReLU1_1', nn.ReLU()),
            ('conv1_2', nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)),
            ('ReLU1_2',nn.ReLU()),
            ('MaxPool1', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('conv2_1',nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)),
            ('ReLU2_1',nn.ReLU()),
            ('conv2_2',nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1)),
            ('ReLU2_2',nn.ReLU()),
            ('MaxPool2', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('conv3_1', nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)),
            ('ReLU3_1',nn.ReLU()),
            ('conv3_2', nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)),
            ('ReLU3_2',nn.ReLU()),
            ('conv3_3', nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)),
            ('ReLU3_3',nn.ReLU()),
            ('MaxPool3', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('conv4_1', nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU4_1',nn.ReLU()),
            ('conv4_2', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU4_2',nn.ReLU()),
            ('conv4_3', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU4_3',nn.ReLU()),
            ('MaxPool4', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('conv5_1', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU5_1',nn.ReLU()),
            ('conv5_2', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU5_2',nn.ReLU()),
            ('conv5_3', nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1)),
            ('ReLU5_3',nn.ReLU()),
            ('MaxPool5', nn.MaxPool2d(kernel_size=2, stride=2,padding=0)),
            ('Flatten6', nn.Flatten()),
            ('fc6', nn.Linear(in_features=25088, out_features=4096)),
            ('ReLU6',nn.ReLU()),
            ('fc7', nn.Linear(in_features=4096, out_features=4096)),
            ('ReLU7',nn.ReLU()),
            ('fc8a', nn.Linear(in_features=4096, out_features=365)),
            ('Softmax8a', nn.Softmax(dim=-1))
        ]
    )
)
torchsummary.summary(vgg16_places, input_size=(3,224,224))

OrderedDict([('conv1_1.weight',
              tensor([[[[ 0.1249, -0.0340, -0.0260],
                        [ 0.1771, -0.1781,  0.0789],
                        [ 0.1609,  0.1315,  0.0441]],
              
                       [[-0.1067,  0.0025,  0.0436],
                        [-0.0574, -0.0933, -0.0364],
                        [ 0.0523,  0.1397, -0.1207]],
              
                       [[ 0.0747, -0.0798,  0.1911],
                        [ 0.0817, -0.0362, -0.0207],
                        [ 0.1303, -0.1805,  0.1081]]],
              
              
                      [[[-0.1197,  0.1410, -0.0985],
                        [-0.1669,  0.1003,  0.1133],
                        [ 0.1640,  0.1650,  0.1618]],
              
                       [[ 0.1654,  0.0763, -0.0461],
                        [ 0.1796,  0.0151, -0.0831],
                        [-0.0343,  0.0287,  0.1841]],
              
                       [[ 0.0955,  0.0835,  0.1785],
                        

In [117]:
vgg16_places.load_state_dict(torch.load('vgg16_places365.caffemodel.pt'))

<All keys matched successfully>

#